In [3]:
import requests
home_url = 'https://eoddata.com/stocklist/TSX.htm'   
response = requests.get(home_url)     
response.status_code

200

In [4]:
page_contents = response.text
len(page_contents)

111343

In [16]:
page_contents[:1000]

'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><link rel="stylesheet" href="../styles/jquery-ui-1.10.0.custom.min.css" type="text/css" /><link rel="stylesheet" href="../styles/main.css" type="text/css" /><link rel="stylesheet" href="../styles/button.css" type="text/css" /><link rel="stylesheet" href="../styles/nav.css" type="text/css" />\r\n  <script src="/scripts/jquery-1.9.0.min.js" type="text/javascript"></script>\r\n  <script src="/scripts/jquery-ui-1.10.0.custom.min.js" type="text/javascript"></script>\r\n\t<script type="text/javascript">\t\tvar _sf_startpt = (new Date()).getTime()</script>\r\n\t<script src="https://js.stripe.com/v3/" type="text/javascript"></script>\r\n\t\r\n\t<script type="text/javascript" src="scripts/jquery-1.4.2.min.js"></script>\r\n<meta name="keywords" content="list of symbols for Toronto Stock Exchange,list of stock sy

In [17]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents, 'html.parser')
type(doc)

bs4.BeautifulSoup

In [18]:
title = doc.find('title')
title

<title>
	List of Symbols for Toronto Stock Exchange [TSX] Starting with A
</title>

In [19]:
tr_parent1 = doc.find_all('tr',{'class':'ro'})
tr_parent2 = doc.find_all('tr',{'class':'re'})
len(tr_parent1)+len(tr_parent2)
td_child1 = tr_parent1[0].find_all('td')
td_child1

[<td><a href="/stockquote/TSX/AAB.htm" title="Display Quote &amp; Chart for TSX,AAB">AAB</a></td>,
 <td>Aberdeen International Inc</td>,
 <td align="right">0.0900</td>,
 <td align="right">0.0900</td>,
 <td align="right">0.0900</td>,
 <td align="right">10,200</td>,
 <td align="right">0.0000</td>,
 <td align="center"><img src="/images/nc.gif"/></td>,
 <td align="left">0.00</td>,
 <td align="right"><a href="/stockquote/TSX/AAB.htm" title="Download Data for TSX,AAB"><img height="14" src="/images/dl.gif" width="14"/></a> <a href="/stockquote/TSX/AAB.htm" title="View Quote and Chart for TSX,AAB"><img height="14" src="/images/chart.gif" width="14"/></a></td>]

In [21]:
import pandas as pd
def parse_document(tr_tag):
    
    td_tag = tr_tag.find_all('td')
    symbol = td_tag[0].find('a').text.strip()
    name = td_tag[1].text.strip()
    high = td_tag[2].text.strip()
    low = td_tag[3].text.strip()
    close = td_tag[4].text.strip()
    volume = td_tag[5].text.strip().replace(',', '')
    url = "http://eoddata.com/" + td_tag[0].find('a')['href']
    
    # Return a dictionary
    return {
        'Symbol': symbol,
        'Name': name,        
        'High': high,
        'Low': low,
        'Close': close,
        'Volume': volume,
        'URL': url
    }  



def write_csv(items, path):
    # Open the file in write mode
    with open(path, 'w') as f:
        # Return if there's nothing to write
        if len(items) == 0:
            return
        
        # Write the headers in the first line
        headers = list(items[0].keys())
        f.write(','.join(headers) + '\n')
        
        # Write one item per line
        for item in items:
            values = []
            for header in headers:
                values.append(str(item.get(header, "")))
            f.write(','.join(values) + "\n")

            
def scrap_stockInfo(alpha_list):  
    base_url = "http://eoddata.com/stocklist/TSX/"
    
    for i in range(len(alpha_list)):
        data_url = base_url + alpha_list[i] +".htm"
        response = requests.get(data_url)
        page_contents = response.text
        doc = BeautifulSoup(page_contents, 'html.parser')
        tr_tags1 = doc.find_all('tr',{'class':'ro'})
        tr_tags2 = doc.find_all('tr',{'class':'re'})
        all_records_1 = [parse_document(tag) for tag in tr_tags1]
        all_records_2 = [parse_document(tag) for tag in tr_tags2]
        all_records = [item for sublist in zip(all_records_1, all_records_2) for item in sublist]
        
        file_name = alpha_list[i] + ".csv"
        write_csv(all_records,file_name)

alpha_list = ['A','B','C','D','E','F','G','H']
scrap_stockInfo(alpha_list)
pd.read_csv('A.csv')


,Symbol,Name,High,Low,Close,Volume,URL
0,AAB,Aberdeen International Inc,0.09,0.09,0.09,57000,http://eoddata.com//stockquote/TSX/AAB.htm
1,AAV,Advantage Oil & Gas Ltd,11.67,11.00,11.53,1352657,http://eoddata.com//stockquote/TSX/AAV.htm
2,ABCT,ABC Technologies Holdings Inc,5.83,5.83,5.83,300,http://eoddata.com//stockquote/TSX/ABCT.htm
3,ABST,Absolute Software Corp,11.69,11.43,11.53,78713,http://eoddata.com//stockquote/TSX/ABST.htm
4,ABTC,Accelerate Carbon Negative Bitcoin ETF,2.96,2.96,2.96,100,http://eoddata.com//stockquote/TSX/ABTC.htm
...,...,...,...,...,...,...,...
115,AX.PR.I,Artis REIT Pref Series I,25.10,25.05,25.05,600,http://eoddata.com//stockquote/TSX/AX.PR.I.htm
116,AX.UN,Artis Real Estate Investment Trust Units,13.10,12.91,13.02,363975,http://eoddata.com//stockquote/TSX/AX.UN.htm
117,AXIS,Axis Auto Finance Inc,0.68,0.67,0.67,50402,http://eoddata.com//stockquote/TSX/AXIS.htm
118,AXU,Alexco Resource Corp,1.02,0.99,1.02,116594,http://eoddata.com//stockquote/TSX/AXU.htm
